# RFSoC Radio Design - BPSK Demonstration
This notebooks presents a BPSK radio system design for the Zynq UltraScale+ RFSoC. The steps outlined in the table of contents below will demonstrate the features and capabilities of the platform.

The BPSK radio system is capable of transmitting and receiving data at 100 kbits/s. The transmitter modulates 8-bit wide fixed point data into BPSK packets for transmission. The transmission system uses a raised cosine to pulse-shape the BPSK data, and then interpolates the samples to 1.024 GSa/s. The pulse-shaped, interpolated BPSK data is mixed with a carrier frequency and transmitted using the RF-DAC. On the receiving end, the RF-ADC also samples at 1.024 GSa/s. The received signal is decimated by a factor of 8 using the RF-ADC's internal half-band lowpass decimators. The signal then passes through a series of decimation stages in the Programmable Logic. Coarse, timing, and carrier synchronisation techniques are then applied to improve signal acquisition. Finally, frame synchronisation using an extended barker sequence is applied through correlation, which allows the BPSK data to be effectively received.

This demonstration provides interactive plotting capabilities so that users can visualise each stage of the BPSK synchronisation process. Users can transmit their own BPSK messages and receive them using a serial terminal. Furthermore, image data can also be transmitted and received using this notebook. Image transmission is accompanied by simple image display widgets for visualisation.

It is recommended that you follow the notebook in order starting with [Overview](#overview).<br><br>,
<b>Table of Contents</b>
1. &nbsp; [Overview](#overview)
2. &nbsp; [The Transmitter](#the_transmitter)
3. &nbsp; [The Receiver](#the_receiver) <br>
    3.1 &nbsp; [Coarse Frequency Synchronisation](#coarse_frequency_synchronisation) <br>
    3.2 &nbsp; [Time Synchronisation](#time_synchronisation) <br>
    3.3 &nbsp; [Frame Synchronisation](#frame_synchronisation) <br>
4. &nbsp; [Transmit and Receive](#transmit_and_receive) <br>
    4.1 &nbsp; [A Simple Message](#a_simple_message) <br>
    4.2 &nbsp; [A Message Repeater](#a_message_repeater) <br>
    4.3 &nbsp; [Image Transfer](#image_transfer) <br>
5. &nbsp; [Summary](#summary)

<div style="background-color: #d9edf7; color: #31708f; padding: 15px;">
    <b>Tip:</b> Make sure you have the SMA loopback cable plugged in as detailed in the <a href="https://github.com/Xilinx/PYNQ_RFSOC_Workshop">instructions here</a>.
</div>

<a class="anchor" id="overview"></a>
## 1. &nbsp; Overview
The aim of this notebook is to demonstrate simple BPSK data transmission using the RFSoC and PYNQ. Two hardware accelerators are provided; one to transmit data, and the other to receive data. Each accelerator is independant of one another and do not communicate. The transmitter modulates 100 kbits/s of data using BPSK, and it interpolates the signal to 1.024 MSa/s with subsequent interpolation stages. The RF-DAC then transmits the data. The receiver is connected to the transmitter using an SMA loopback cable. The RF-ADC will initially decimate the data and the receiver hardware accelerator will be responsible for synchronising to the signal and extracting the modulated data.

A Direct Memory Access (DMA) controller is provided to transfer data from Jupyter Labs to the transmitter. Another DMA is provided to transfer data from the receiver to Jupyter Labs. Jupyter will be used to control the hardware accelerators and inspect the receiver's synchronisation stages. The entire system can be seen in the figure below:

<br>

<img src="assets/system_overview.png" style="width: 70%;" align="middle"/>

<br>

<a class="anchor" id="the_transmitter"></a>
## 2. &nbsp; The Transmitter

<br>

<img src="assets/transmitter.png" style="width: 90%;" align="middle"/>

<br>

<a class="anchor" id="the_receiver"></a>
## 3. &nbsp; The Receiver

<br>

<img src="assets/receiver.png" style="width: 90%;" align="middle"/>

<br>

In [ ]:
from rfsoc_radio_bpsk import BpskRadio

radio = BpskRadio(init_rf_clks = True)

When the hardware system was designed, a data inspector was added that will allow you to transfer frames of data from the Programmable Logic to the external Processing System memory. Software has been developed that manipulates the samples of data for visualisation using the Plotly Python library. You can try this for yourself by running the code cell below.

Upon executing the cell you will be presented with time domain, constellation, and frequency spectrum plots. These plots can be continuously updated by clicking the start button. You are able to visualise other points in the receiver by using the observation point dropdown menu.

In [ ]:
radio.receiver.visualise()

Right-click the area above, and in the drop-down menu that appears, select "Create New View for Output". This action will move the plots to another window in Jupyter Labs, allowing you to scroll further down the notebook while still being able to visualise and interact with the plots.

There is also one more tool you will require while interacting with this notebook, a radio dashboard. This will allow you to change the mixer frequencies of the RF-DAC and RF-ADC. You can also choose to switch-off parts of the radio system such as the transmitter, and various parts of the receiver. You can load the radio dashboard by running the code cell below. After you have executed the cell, you should right click the output area, and select "Create New View for Output" from the drop-down menu. Again, this will allow you to interact with the remainder of the notebook and retain access to the radio dashboard.

In [ ]:
radio.dashboard()

<a class="anchor" id="coarse_frequency_synchronisation"></a>
### 3.1 &nbsp; Coarse Frequency Synchronisation

There are no code cells to run in this section. We will use the output plots and radio dashboard to interact with our system. Using our inspector tool, set the observation point to "CIC Decimator" using the drop-down menu. Take a moment to inspect the time domain and frequency spectrum plots. Notice that the time domain signal is smooth and the pulse shaped symbols can be easily inspected. While the frequency spectrum has a primary peak directly on 0 Hz. The system is correctly synchronised in terms of mixer frequency.

Coarse frequency synchronisation corrects frequency offset in the received signal, which may be due to incorrect tuning or channel effects. We can evaluate incorrect tuning by setting our RF-ADC mixer to a frequency that is not the same as the RF-DAC. Try this now by using the radio dashboard to set the RF-ADC frequency to 65.25. 

Using our inspector tool, we can visualise the effects incorrect tuning can have on our signal in the time and frequency domain. Ensure that the observation point is set to CIC Decimator. Notice, that the waveform now oscillates over symbols in the time domain plot. In the frequency spectrum, the peak is no longer located at 0 Hz, and has shifted by 1.25 MHz.

The coarse frequency synchronisation module in our hardware receiver can compensate for incorrect frequency tuning. This can be visualised by changing the observation point to "Coarse Frequency Synchronisation" and evaluating the output time domain and frequency spectrum plots. Notice that the time domain signal no longer oscillates over received symbols, and the primary peak is now located over 0 Hz.

The hardware architecture of the coarse frequency synchronisation module is located in:

<div style="background-color: #d9edf7; color: #31708f; padding: 15px;">
    <b>System Generator:</b> &nbsp; ...models/bpsk_receiver/bpsk_rx_coarse_sync.slx.
</div>

You should take a moment to inspect this model and understand its contents.

<a class="anchor" id="time_synchronisation"></a>
### 3.2 &nbsp; Time and Carrier Synchronisation

Upon coarsely synchronising to our received signal, we now need to determine where its maximum effect points are and use these to retrieve the modulated data. Time synchronisation is performed by evaluating the point in a signal that contains the transmitted data. In this system, time synchronisation is perfomed by first determining the slope of the received waveform and adjusting the timing error appropriately. This architecture can be inspected in the following model:

<div style="background-color: #d9edf7; color: #31708f; padding: 15px;">
    <b>System Generator:</b> &nbsp; ...models/bpsk_receiver/bpsk_rx_time_sync.slx.
</div>

Using the inspection tool, we can visualise the effects of correct time synchronisation. Set the observation point to "Time Synchronisation" and inspect the time domain and constellation plots. You will be able to clearly see that the time domain plot is shaped like a saw tooth. This is because the time synchronisation algorithm is selecting the most effective points to sample and retrieve data. In the constellation plot, a BPSK signal can be seen, that is rotating in a circular motion. The rotation occurs because the real and imaginary components of the signal are constantly changing in amplitude. The amplitude change occurs as the signal is not carrier synchronised (or phase aligned correctly).

Carrier synchronisation can be performed by determining the angle of error between the expected phase and the received phase. BPSK modulation expects to only have a signal with a real component. Therefore, we must rotate the constellation appropriately to maximise the real component amplitude and suppress the imaginary component. This will cause the BPSK constellation to lie on the x-axis if the constellation plot. The architecture of the carrier synchronisation module can be seen in:

<div style="background-color: #d9edf7; color: #31708f; padding: 15px;">
    <b>System Generator:</b> &nbsp; ...models/bpsk_receiver/bpsk_rx_phase_sync.slx.
</div>

The inspection tool can be used to visualise the effect of carrier synchronisation. Change the signal of the inspection to "Phase Synchronisation" using the observation point drop-down menu. You will be able to inspect the constellation plot and time domain plot. Notice that the constellation has stopped rotating and the time domain signal has stabilised.

<a class="anchor" id="frame_synchronisation"></a>
### 3.3 &nbsp; Frame Synchronisation

The frame synchronisation architecture allows a transmitted signal, which contains a sequence of known bits, to be recognised. Before the transmission of data, a sequence of known bits are inserted at the start of a packet and are recognised at the receiver. When the receiver locks on to the sequence of known bits, it begins to read the remainder of the data.

The 'known bits' in this radio architecture are barker codes. These are unique codes that result in maximum correlation when the reference and received sequence are directly aligned. They are particularly unique and enable receivers to effectively identify incoming data packets.

<div style="background-color: #d9edf7; color: #31708f; padding: 15px;">
    <b>System Generator:</b> &nbsp; ...models/bpsk_receiver/bpsk_rx_frame_sync.slx.
</div>

<a class="anchor" id="transmit_and_receive"></a>
## 4. &nbsp; Transmit and Receive

<a class="anchor" id="a_simple_message"></a>
### 4.1 &nbsp; A Simple Message

In [ ]:
# Import Terminal() from the quick widgets library
from quick_widgets import Terminal
import numpy as np

# Create a text terminal for interacting with received data
terminal = Terminal(description='Received Message:')

# Do we want to debug?
debug = False

# Create a custom callback function that is executed when the receiver
# interrupt is triggered
def terminal_callback():
    global debug
    frame = radio.receiver.frame
    payload = np.where(frame["payload"] > 127, 0, frame["payload"]).tostring().decode('ascii')
    if debug:
        data = 'Header: ' + str({i:frame[i] for i in frame if i!="payload"}) \
        + '\rPayload: ' + payload + '\r\r'
    else:
        data = payload
    terminal.append(data)
    
# Set the terminal_callback function as the callback for the receiver
# when an interrupt is triggered
radio.receiver.monitor.callback = [terminal_callback]

# Get the widget for interaction with received data
terminal.get_widget()

In [ ]:
radio.transmitter.data('Hello World!\r')
radio.transmitter.start()

In [ ]:
# Do we want to debug?
debug = True
radio.transmitter.data('The quick brown fox jumps over the lazy dog.' + '\r' +
                          'How razorback-jumping frogs can level six piqued gymnasts.' + '\r' +
                          'Pack my box with five dozen liquor jugs.' + '\r' +
                          'Jackdaws love my big sphinx of quartz.' + '\r' +
                          'Amazingly few discotheques provide jukeboxes.' + '\r' +
                          'Now fax quiz Jack! my brave ghost pled.' + '\r')
radio.transmitter.start()

<a class="anchor" id="a_message_repeater"></a>
### 4.2 &nbsp; A Message Repeater

In [ ]:
counter = 0
debug = False

def message_callback():
    global counter
    message = 'Hello World! ' + str(counter) + '\r'
    radio.transmitter.data(message)
    counter += 1
    
# Set the message_callback function as the callback before transmitting
# data using the transmitter
radio.transmitter.monitor.callback = [message_callback]

# Start the transmitter in repeat mode
radio.transmitter.mode = 'repeat'

# Set the transmission rate to half a second
radio.transmitter.monitor.rate = 0.5

radio.transmitter.start()

In [ ]:
radio.transmitter.stop()

<a class="anchor" id="image_transfer"></a>
### 4.3 &nbsp; Image Transfer

In [ ]:
# Import ImageViewer() from the quick widgets library
from quick_widgets import ImageViewer
import numpy as np
import ipywidgets as ipw

# Open the target image using bytes
image = []
for i in range(4):
    file = open('assets/small_cat_' + str(i) + '.jpg', "rb")
    image.append(file.read())

# Set flip variable
counter = 0

# Create image viewer for transmitted image
sendimage = ImageViewer(description='Transmitted Image')

def sendimage_callback():
    global counter
    global image
    radio.transmitter.data(image[counter])
    sendimage.update(image[counter])
    if counter > 2:
        counter = 0
    else:
        counter += 1
    
# Set the sendimage_callback function as the callback before transmitting
# data using the transmitter
radio.transmitter.monitor.callback = [sendimage_callback]

# Slow down the transmission rate to two seconds
radio.transmitter.monitor.rate = 2.5

sendimage.update(image[0])

# Get the widget for interaction with sent data
ipw.HBox([sendimage.get_widget()])

In [ ]:
# Import ImageViewer() from the quick widgets library
from quick_widgets import ImageViewer
import numpy as np
import ipywidgets as ipw

# Create a receiver buffer to store packets of data
recvbuffer = np.empty(0, dtype=np.uint8)

# Create an image viewer object for visualising the received data
recvimage = ImageViewer(description='Received Image')

# Create a custom callback function that is executed when the receiver
# interrupt is triggered
def recvimage_callback():
    global recvbuffer
    frame = radio.receiver.frame
    payload = frame["payload"]
    if ((frame["flags"] >> 1) & 1):
        recvbuffer = np.array(payload, dtype=np.uint8)
    else:
        recvbuffer = np.append(recvbuffer, payload)
    if (frame["flags"] & 1):
        recvimage.update(recvbuffer.tobytes())
        
# Set the terminal_callback function as the callback for the receiver
# when an interrupt is triggered
radio.receiver.monitor.callback = [recvimage_callback]

# Get the widget for interaction with received data
ipw.HBox([recvimage.get_widget()])

In [ ]:
# Start the transfer
radio.transmitter.mode = 'repeat'
radio.transmitter.start()

In [ ]:
radio.transmitter.stop()

<a class="anchor" id="summary"></a>
## 5. &nbsp; Summary